# Convert Stirling Council's waste-management dataset to an RDF data cube

This experiment takes (non-linked) data from Stirling Council's [waste-management dataset](https://data.stirling.gov.uk/dataset/waste-management) and converts it into an [RDF data cube](https://www.w3.org/TR/vocab-data-cube/).


In [19]:
; Add code libraries.

%classpath add mvn org.clojure data.csv 1.0.0
(require '[clojure.data.csv :as csv])

(require '[clojure.java.io :as io])
(require '[clojure.pprint :as pp])
(require '[clojure.edn :as edn])
(require '[clojure.string :as str])

(import 'java.time.LocalDate)
(import 'com.twosigma.beakerx.chart.xychart.TimePlot
        'com.twosigma.beakerx.chart.xychart.plotitem.Line)

class com.twosigma.beakerx.chart.xychart.plotitem.Line

In [2]:
; Define convenience functions.

; Convert the CSV structure to a list-of-maps structure.
(defn to-maps [csv-data]
    (map zipmap (->> (first csv-data)
                    ;(map keyword)
                    repeat)
                (rest csv-data)))

; Convert the string to be URI friendly.
(defn slugize [s]
  (-> s
      str
      (str/replace #"[^\w/]" "-")
      (str/replace #"-+" "-")
      (str/replace #"-$" "")
      str/lower-case))

#'beaker_clojure_shell_0497c17c-bbae-4f91-95bf-c8e04e1cdd6b/slugize

Fetch the source data:
* [20200406-waste-management-january-to-march-2020.csv](https://data.stirling.gov.uk/dataset/a1768130-ff1b-49ef-a98d-e474ab8a94e0/resource/22683cc9-d7fc-4a87-8001-c9dc4e4e4a62/download/20200406-waste-management-january-to-march-2020.csv)
* [20200116-waste-management-january-to-december-2019.csv](https://data.stirling.gov.uk/dataset/a1768130-ff1b-49ef-a98d-e474ab8a94e0/resource/d5fa214b-1cd3-4b4d-9624-f91e302ba1d2/download/20200116-waste-management-january-to-december-2019.csv)
* [20190322-waste-management-january-to-december-2018.csv](https://data.stirling.gov.uk/dataset/a1768130-ff1b-49ef-a98d-e474ab8a94e0/resource/1dc20298-7291-4937-ad19-6e66566b8d6c/download/20190322-waste-management-january-to-december-2018.csv)


In [3]:
; Read the source data

(def source-files [
    "20200406-waste-management-january-to-march-2020.csv"
    "20200116-waste-management-january-to-december-2019.csv"
    "20190322-waste-management-january-to-december-2018.csv"])

; Read, parse & merge the source-files
(def source-data
    (->> source-files
        (map (fn [source-file]
                (to-maps
                    (with-open [reader (io/reader source-file)]
                        (doall
                            (csv/read-csv reader))))))
        flatten))
                
(println (count source-data) "rows")

; Sample
(repeatedly 10 #(rand-nth source-data))

16365 rows


We're going to use Swirrl IT's [table2qb tool](https://github.com/Swirrl/table2qb) (build from its `master` branch on 6th May 2020) to do most of the conversion work.



In [4]:
; Define the data's components - i.e. its dimensions, measures and attributes.

; The Date dimension uses an existing vocabulary.
; The Measure Type dimension is defined by data cube specification. 
; So, the Type and Route dimensions and the Count measure need to be defined.
(def components [
    ["Label"            "Description"                        "Component Type"  "Codelist"]
    ; -----             -----------                          --------------    --------
    ["Type"             "The type of waste"                  "Dimension"       "http://data-commons-scotland.org/def/concept-scheme/type"]
    ["Route"            "The collection route"               "Dimension"       "http://data-commons-scotland.org/def/concept-scheme/route"]
    ["Count"            "The quantity of waste"              "Measure"         ""]])

; Store in a CSV file             
(with-open [writer (io/writer "components.csv")]
    (csv/write-csv writer components))  

null

In [5]:
; Define the data's codelist of waste types.

(def codelist-header-row ["Label" "Notation" "Parent Notation" "Description"])

(def type-codelist
    (->> source-data
        (map #(get % "Waste"))
        distinct
        (map #(vector % (slugize %) "all" ""))
        (cons ["All" "all" "" ""])))
        
(println (count type-codelist) "rows")

(->> type-codelist
    (map #(zipmap codelist-header-row %)))

10 rows


In [6]:
; Define the data's codelist of collection routes.

(def route-codelist
    (->> source-data
        (map #(get % "Account"))
        distinct
        (map #(vector % (slugize %) "" ""))
        (cons ["All" "all" "" ""])))
        
(println (count route-codelist) "rows")

(->> route-codelist
    (map #(zipmap codelist-header-row %)))

34 rows


In [7]:
; Define the data's codelist of units of measure.

(def units-codelist [
    ["All" "all" "" ""]
    ["Tonnes" "tonnes" "" ""]])
        
(println (count units-codelist) "rows")

(->> units-codelist
    (map #(zipmap codelist-header-row %)))

2 rows


In [8]:
; Store the codelists in CSV files

(doseq [name ["type" "route" "units"]]
    (with-open [writer (io/writer (str name ".csv"))]
        (csv/write-csv writer 
            (cons codelist-header-row 
                  (-> name (str "-codelist") symbol resolve var-get)))))

null

In [9]:
; Tidy-up the data's observations (see http://vita.had.co.nz/papers/tidy-data.pdf)

(def obs-header-row ["Date" "Type" "Route" "Measure Type" "Unit" "Value"])

(def obs
    (->> source-data
        (map #(vector ; with the structure [Date Type Route Value]
            (let [s (get % "Date")] (str (subs s 6 10) "-" (subs s 3 5) "-" (subs s 0 2))) ; ignore the time component 
            (slugize (get % "Waste"))
            (slugize (get % "Account"))
            (get % "Quantity")))
        (group-by (juxt first second #(nth % 2))) ; group by [Date Type Route]
        (map (fn [[[k0 k1 k2] l]] (vector k0 k1 k2 (->> l (map #(get % 3)) (map edn/read-string) (apply +))))) ; sum the Value values in each group
        (map (fn [[v0 v1 v2 v3]] (vector v0 v1 v2 "Count" "Tonnes" v3))))) ; decorate
        
(println (count obs) "rows")

; Sample
(repeatedly 10 
    #(rand-nth 
        (map (fn [data-row] (zipmap obs-header-row data-row)) obs)))

10020 rows


In [10]:
; Store the tidied-up observations in a CSV file

(with-open [writer (io/writer "obs.csv")]
    (csv/write-csv writer 
        (cons obs-header-row 
              obs)))

null

In [11]:
; Specify how the observations should be processed

(def columns [
    ["title"         "name"         "component_attachment"  "property_template"                                            "value_template"                                              "datatype" "value_transformation"]
    ; -----           ----           --------------------    -----------------                                              --------------                                                --------   --------------------
    ["Date"          "date"         "qb:dimension"          "http://purl.org/linked-data/sdmx/2009/dimension#refPeriod"    "http://reference.data.gov.uk/id/day/{date}"                  "string"   ""]
    ["Type"          "type"         "qb:dimension"          "http://data-commons-scotland.org/def/dimension/type"          "http://data-commons-scotland.org/def/concept/type/{type}"    "string"   "slugize"]
    ["Route"         "route"        "qb:dimension"          "http://data-commons-scotland.org/def/dimension/route"         "http://data-commons-scotland.org/def/concept/route/{route}"  "string"   "slugize"]
    ["Measure Type"  "measure_type" "qb:dimension"          "http://purl.org/linked-data/cube#measureType"                 "http://data-commons-scotland.org/def/measure/{measure_type}" "string"   "slugize"]
    ["Unit"          "unit"         "qb:attribute"          "http://purl.org/linked-data/sdmx/2009/attribute#unitMeasure"  "http://data-commons-scotland.org/def/concept/measurement-units/{unit}" "string"   "unitize"]
    ["Value"         "value"        ""                      "http://data-commons-scotland.org/def/measure/{measure_type}"  ""                                                            "number"   ""]
    ["Count"         "count"        "qb:measure"            "http://data-commons-scotland.org/def/measure/count"           ""                                                            "number"   ""]])

; Store in a CSV file             
(with-open [writer (io/writer "columns.csv")]
    (csv/write-csv writer columns))

null

Use `table2qb`'s _pipeline_ commands to generate RDF files...

```
./table2qb exec components-pipeline --input-csv components.csv --base-uri http://data-commons-scotland.org/ --output-file components.ttl

./table2qb exec codelist-pipeline --codelist-csv type.csv --codelist-name Type --codelist-slug type --base-uri http://data-commons-scotland.org/ --output-file type.ttl

./table2qb exec codelist-pipeline --codelist-csv route.csv --codelist-name Route --codelist-slug route --base-uri http://data-commons-scotland.org/ --output-file route.ttl

./table2qb exec codelist-pipeline --codelist-csv units.csv --codelist-name "Measurement Units" --codelist-slug "measurement-units" --base-uri http://data-commons-scotland.org/ --output-file measurement-units.ttl

./table2qb exec cube-pipeline --input-csv obs.csv --dataset-name "Stirling Waste Management" --dataset-slug "stirling-waste-management" --column-config columns.csv --base-uri http://data-commons-scotland.org/ --output-file cube.ttl
```

Load those RDF files into in a triplestore to create a graph
and try the following SPARQL query against it:
```
prefix qb: <http://purl.org/linked-data/cube#>
select (count(distinct ?obs) AS ?obs_count) WHERE {
  ?obs qb:dataSet <http://data-commons-scotland.org/data/stirling-waste-management> .
}
```
That query ought to return `10020`, i.e. the number of rows in the `obs` data collection that we constructed above.



Now let's query for the quantity (in tonnes) of waste collected on a specific day (20 Feb 2020), of a specific waste type (01 Household...) and a specific collection route (504 Route 4):
```
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix sdmx: <http://purl.org/linked-data/sdmx/2009/dimension#>
prefix day: <http://reference.data.gov.uk/id/day/>
prefix dcsd: <http://data-commons-scotland.org/def/dimension/>
prefix dcsm: <http://data-commons-scotland.org/def/measure/>
select ?tonnes where {
  ?obs
	sdmx:refPeriod day:2020-02-20 ;
	dcsd:type/rdfs:label "01 Household to Avondale" ;
	dcsd:route/rdfs:label "504 Route 4 (NEW )" ;
	dcsm:count ?tonnes .
}
```
That query ought to return `18.98`, i.e. the sum of the Quantity values of the following two rows from `source-data`:

In [12]:
(->> source-data
    (filter #(-> % (get "Date") (str/starts-with? "20/02/2020")))
    (filter #(-> % (get "Waste") (= "01 Household to Avondale")))
    (filter #(-> % (get "Account") (= "504 Route 4 (NEW )"))))

Now let's execute a SPARQL query that will find the total tonnes of waste collected per route per day:
```
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix sdmx: <http://purl.org/linked-data/sdmx/2009/dimension#>
prefix dcsd: <http://data-commons-scotland.org/def/dimension/>
prefix dcsm: <http://data-commons-scotland.org/def/measure/>

select 
  ?route 
  (strafter(str(?day_uri), "http://reference.data.gov.uk/id/day/") as ?day) 
  (sum(?tonnes_per_type) as ?tonnes) 

where {
  ?obs
    sdmx:refPeriod ?day_uri ;
    dcsd:type/rdfs:label ?type ;
    dcsd:route/rdfs:label ?route ;
    dcsm:count ?tonnes_per_type .
    
} group by ?route ?day_uri 
```
I've copied the output of that query into the file: tonnes-collected-per-route-per-day.csv

In [17]:
(def tonnes-per-route-per-day
    (to-maps
        (with-open [reader (io/reader "tonnes-collected-per-route-per-day.csv")]
            (doall
                (csv/read-csv reader)))))
                            
(println (count tonnes-per-route-per-day) "rows")

; Sample
(repeatedly 10 #(rand-nth tonnes-per-route-per-day))

9423 rows


In [22]:
; Plot an info-graphic of the tonnes of waste collected per route per day

(def lines
    (->> tonnes-per-route-per-day
        (group-by #(get % "route"))
        (map (fn [[route coll1]]
                (let [coll2 (sort-by #(get % "day") coll1)]
                    {:label route
                     :x-series (map #(LocalDate/parse (get % "day")) coll2)
                     :y-series (map #(get % "tonnes") coll2)})))))

(def time-plot
    (doto (TimePlot.)
        (.setTitle "Tonnes of waste collected per route per day")
        (.setXLabel "Day")
        (.setYLabel "Tonnes of waste collected")))
(doseq [line lines]
    (.add time-plot (doto (Line.)
                        (.setDisplayName (:label line))
                        (.setX (:x-series line))
                        (.setY (:y-series line)))))
time-plot